In [18]:
%%capture
!pip3 install --no-cache-dir --upgrade git+https://github.com/evfro/polara.git@develop#egg=polara

In [2]:
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

import polara
from polara import get_movielens_data
from polara.preprocessing.dataframes import leave_one_out, reindex

from dataprep import transform_indices
from evaluation import topn_recommendations, downvote_seen_items

from polara.lib.tensor import hooi

from sa_hooi import sa_hooi, form_attention_matrix, get_scaling_weights, generate_position_projector

from polara.lib.sparse import tensor_outer_at
from scipy.sparse import csr_matrix
from scipy.linalg import solve_triangular, sqrtm

%load_ext autoreload
%autoreload 2

In [3]:
def full_preproccessing():
    data = get_movielens_data(include_time=True)
    test_timepoint = data['timestamp'].quantile(
    q=0.8, interpolation='nearest'
    )

    test_data_ = data.query('timestamp >= @test_timepoint')
    train_data_ = data.query(
    'userid not in @test_data_.userid.unique() and timestamp < @test_timepoint'
    )
    
    training, data_index = transform_indices(train_data_.copy(), 'userid', 'movieid')
    test_data = reindex(test_data_, data_index['items'])

    testset_, holdout_ = leave_one_out(
    test_data, target='timestamp', sample_top=True, random_state=0
    )
    testset_valid_, holdout_valid_ = leave_one_out(
        testset_, target='timestamp', sample_top=True, random_state=0
    )

    test_users_val = np.intersect1d(testset_valid_.userid.unique(), holdout_valid_.userid.unique())
    testset_valid = testset_valid_.query('userid in @test_users_val').sort_values('userid')
    holdout_valid = holdout_valid_.query('userid in @test_users_val').sort_values('userid')

    test_users = np.intersect1d(testset_.userid.unique(), holdout_.userid.unique())
    testset = testset_.query('userid in @test_users').sort_values('userid')
    holdout = holdout_.query('userid in @test_users').sort_values('userid')


    assert holdout_valid.set_index('userid')['timestamp'].ge(
        testset_valid
        .groupby('userid')
        ['timestamp'].max()
    ).all()

    data_description = dict(
        users = data_index['users'].name,
        items = data_index['items'].name,
        feedback = 'rating',
        n_users = len(data_index['users']),
        n_items = len(data_index['items']),
        n_ratings = training['rating'].nunique(),
        min_rating = training['rating'].min()
    )

    return training, testset_valid, holdout_valid, testset, holdout, data_description

In [4]:
#data = RecommenderData(training, "userid", "movieid", "rating")
#data.prpeare_training_only()

#data.set_test_data(testset=testset_valid, holdout=holdout_valid)

#model.evaluate(switch_positive=4)


In [5]:
training, testset_valid, holdout_valid, testset, holdout, data_description = full_preproccessing()

Filtered 177 invalid observations.


In [14]:
from IPython.utils import io
from scipy.special import softmax

def tf_model_build(config, data, data_description, testset, holdout, attention_matrix=np.array([])):
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    idx = data[[userid, itemid, feedback]].values
    idx[:, -1] = idx[:, -1] - data_description['min_rating'] # works only for integer ratings!
    val = np.ones(idx.shape[0], dtype='f8')
    
    n_users = data_description["n_users"]
    n_items = data_description["n_items"]
    n_ratings = data_description["n_ratings"]
    shape = (n_users, n_items, n_ratings)
    core_shape = config['mlrank']
    num_iters = config["num_iters"]
    
    if (attention_matrix.shape[0] == 0):
        attention_matrix = form_attention_matrix(
            data_description['n_ratings'],
            **config['params'],
            format = 'csr'
        )
        
    attention_matrix = np.array(attention_matrix)

    item_popularity = (
        data[itemid]
        .value_counts(sort=False)
        .reindex(range(n_items))
        .fillna(1)
        .values
    )
    scaling_weights = get_scaling_weights(item_popularity, scaling=config["scaling"])

    with io.capture_output() as captured:
        u0, u1, u2 = sa_hooi(
            idx, val, shape, config["mlrank"],
            attention_matrix = attention_matrix,
            scaling_weights = scaling_weights,
            testset = testset,
            holdout = holdout,
            data_description = data_description,
            max_iters = config["num_iters"],
            parallel_ttm = True,
            randomized = config["randomized"],
            growth_tol = config["growth_tol"],
            seed = config["seed"],
            iter_callback = None,
        )
    
    return u0, u1, u2, attention_matrix    
    
config = {
    "scaling": 1,
    "mlrank": (30, 30, 5),
    "n_ratings": data_description['n_ratings'],
    "num_iters": 5,
    "params": None,
    "randomized": True,
    "growth_tol": 1e-4,
    "seed": 42
}

def tf_scoring(params, data, data_description, context=["3+4+5"]):
    user_factors, item_factors, feedback_factors, attention_matrix = params
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    data = data.sort_values(userid)
    useridx = data[userid]
    itemidx = data[itemid].values
    ratings = data[feedback].values
    ratings = ratings - data_description['min_rating']
    
    n_users = useridx.nunique()
    n_items = data_description['n_items']
    n_ratings = data_description['n_ratings']
    
    inv_attention = solve_triangular(attention_matrix, np.eye(5), lower=True)
    
    tensor_outer = tensor_outer_at('cpu')
    #matrix_softmax = softmax(inv_attention.T @ feedback_factors)
    matrix_softmax = inv_attention.T @ feedback_factors
    #
    if (context == "5"): # make softmax 
        inv_aT_feedback = matrix_softmax[-1, :]
    elif (context == "4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[-2:, :], axis=0)
    elif (context == "3+4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[-3:, :], axis=0)
    #elif (context == "2+3+4+5"):
    #    inv_aT_feedback = np.sum(matrix_softmax[-4:, :], axis=0)
    elif (context == "3+4+5-2-1"):
        inv_aT_feedback = np.sum(matrix_softmax[-3:, :], axis=0) - np.sum(matrix_softmax[:2, :], axis=0)
        
    scores = tensor_outer(
        1.0,
        item_factors,
        attention_matrix @ feedback_factors,
        itemidx,
        ratings
    )
    scores = np.add.reduceat(scores, np.r_[0, np.where(np.diff(useridx))[0]+1]) # sort by users
    scores = np.tensordot(
        scores,
        inv_aT_feedback,
        axes=(2, 0)
    ).dot(item_factors.T)

    return scores

def model_evaluate(recommended_items, holdout, holdout_description, alpha=3, topn=10, dcg=False):
    itemid = holdout_description['items']
    rateid = holdout_description['feedback']
    holdout_items = holdout[itemid].values
    n_test_users = recommended_items.shape[0]
    assert recommended_items.shape[0] == len(holdout_items)
    
    hits_mask = recommended_items[:, :topn] == holdout_items.reshape(-1, 1)
    pos_mask = (holdout[rateid] >= alpha).values
    neg_mask = (holdout[rateid] < alpha).values
    
    # HR calculation
    #hr = np.sum(hits_mask.any(axis=1)) / n_test_users
    hr_pos = np.sum(hits_mask[pos_mask].any(axis=1)) / n_test_users
    hr_neg = np.sum(hits_mask[neg_mask].any(axis=1)) / n_test_users
    hr = hr_pos + hr_neg
    
    # MRR calculation
    hit_rank = np.where(hits_mask)[1] + 1.0
    mrr = np.sum(1 / hit_rank) / n_test_users
    pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
    mrr_pos = np.sum(1 / pos_hit_rank) / n_test_users
    neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
    mrr_neg = np.sum(1 / neg_hit_rank) / n_test_users
    
    # Matthews correlation
    TP = np.sum(hits_mask[pos_mask]) # + 
    FP = np.sum(hits_mask[neg_mask]) # +
    cond = (hits_mask.sum(axis = 1) == 0)
    FN = np.sum(cond[pos_mask])
    TN = np.sum(cond[neg_mask])
    N = TP+FP+TN+FN
    S = (TP+FN)/N
    P = (TP+FP)/N
    C = (TP/N - S*P) / np.sqrt(P*S*(1-P)*(1-S))
    
    # DCG calculation
    if dcg:
        pos_hit_rank = np.where(hits_mask[pos_mask])[1] + 1.0
        neg_hit_rank = np.where(hits_mask[neg_mask])[1] + 1.0
        ndcg = np.mean(1 / np.log2(pos_hit_rank+1))
        ndcl = np.mean(1 / np.log2(neg_hit_rank+1))
    
    # coverage calculation
    n_items = holdout_description['n_items']
    cov = np.unique(recommended_items).size / n_items
    if dcg:
        return hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C, ndcg, ndcl
    else:
        return hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C

def make_prediction(tf_scores, holdout, data_description, mode, context="", print_mode=True):
    if (mode and print_mode):
        print(f"for context {context} evaluation ({mode}): \n")
    for n in [5, 10, 20]:
        tf_recs = topn_recommendations(tf_scores, n)
        hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C = model_evaluate(tf_recs, holdout, data_description, topn=n)
        if (print_mode):
            print(f"HR@{n} = {hr:.4f}, MRR@{n} = {mrr:.4f}, Coverage@{n} = {cov:.4f}")
            print(f"HR_pos@{n} = {hr_pos:.4f}, HR_neg@{n} = {hr_neg:.4f}")
            print(f"MRR_pos@{n} = {mrr_pos:.4f}, MRR_neg@{n} = {mrr_neg:.4f}")
            print(f"Matthews@{n} = {C:.4f}")
            print("-------------------------------------")
        if (n == 10):
            mrr10 = mrr
            hr10 = hr
            c10 = C
    return mrr10, hr10, c10

def valid_mlrank(mlrank):
    '''
    Only allow ranks that are suitable for truncated SVD computations
    on unfolded compressed tensor (the result of ttm product in HOOI).
    '''
    #s, r1, r2, r3 = mlrank
    s, r1, r3 = mlrank
    r2 = r1
    #print(s, r1, r2, r3)
    return r1*r2 > r3 and r1*r3 > r2 and r2*r3 > r1

In [15]:
from tqdm import tqdm 
from polara.evaluation.pipelines import random_grid

def full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix):

    config["mlrank"] = (30, 30, 5)
    print("Starting pipeline...")
    print("Training with different context in progress...")
    print("------------------------------------------------------")

    for context in ["5", "4+5", "3+4+5", "3+4+5-2-1"]:
        tf_params = tf_model_build(config, training, data_description, testset_valid, holdout_valid, attention_matrix=attention_matrix)
        seen_data = testset_valid
        tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
        downvote_seen_items(tf_scores, seen_data, data_description)
        cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout_valid, data_description, "Validation", context)
        print("------------------------------------------------------")

    print(f"Tuning model for all contexts...\n")

    rank_grid = []
    for i in range(5, 10):
        rank_grid.append(2 * 2 ** i)
        rank_grid.append(3 * 2 ** i)
    
    rank_grid = np.array(rank_grid)
    tf_hyper = {
    'scaling': np.linspace(0, 2, 21),
    'r1': rank_grid #np.arange(100, 220, 25),
    #'r2': np.arange(50, 801, 25),
    'r3': range(2, 6, 1),
    }

    grid, param_names = random_grid(tf_hyper, n=0)
    tf_grid = [tuple(mlrank) for mlrank in grid if valid_mlrank(mlrank)]

    hr_tf = {}
    hr_pos_tf = {}
    hr_neg_tf = {}
    mrr_tf = {}
    mrr_pos_tf = {}
    mrr_neg_tf = {}
    cov_tf = {}
    C_tf = {}
    
    seen_data = testset_valid
    
    for mlrank in tqdm(tf_grid):
        with io.capture_output() as captured:
            r1, r3 = mlrank[1:]
            cur_mlrank = tuple((r1, r1, r3))
            config['mlrank'] = cur_mlrank
            config['scaling'] = mlrank[0]
            tf_params = tf_model_build(config, training, data_description, testset_valid, holdout_valid, attention_matrix=attention_matrix)
            for context in ["5", "4+5", "3+4+5", "3+4+5-2-1"]:
                tf_scores = tf_scoring(tf_params, seen_data, data_description, context)
                downvote_seen_items(tf_scores, seen_data, data_description)
                tf_recs = topn_recommendations(tf_scores, topn=10)
                
                hr, hr_pos, hr_neg, mrr, mrr_pos, mrr_neg, cov, C = model_evaluate(tf_recs, holdout_valid, data_description, topn=10)
                hr_tf[(context, cur_mlrank, mlrank[0])] = hr
                hr_pos_tf[(context, cur_mlrank, mlrank[0])] = hr_pos
                hr_neg_tf[(context, cur_mlrank, mlrank[0])] = hr_neg
                mrr_tf[(context, cur_mlrank, mlrank[0])] = mrr
                mrr_pos_tf[(context, cur_mlrank, mlrank[0])] = mrr_pos
                mrr_neg_tf[(context, cur_mlrank, mlrank[0])] = mrr_neg
                cov_tf[(context, cur_mlrank, mlrank[0])] = cov
                C_tf[(context, cur_mlrank, mlrank[0])] = C

    print(f'Best HR={pd.Series(hr_tf).max():.4f} achieved with context {pd.Series(hr_tf).idxmax()[0]} and mlrank = {pd.Series(hr_tf).idxmax()[1]} and scale factor = {pd.Series(hr_tf).idxmax()[2]}')
    print(f'Best HR_pos={pd.Series(hr_pos_tf).max():.4f} achieved with context {pd.Series(hr_pos_tf).idxmax()[0]} and mlrank = {pd.Series(hr_pos_tf).idxmax()[1]} and scale factor = {pd.Series(hr_pos_tf).idxmax()[2]}')
    print(f'Best HR_neg={pd.Series(hr_neg_tf).min():.4f} achieved with context {pd.Series(hr_neg_tf).idxmin()[0]} and mlrank = {pd.Series(hr_neg_tf).idxmin()[1]} and scale factor = {pd.Series(hr_neg_tf).idxmin()[2]}')
    
    print(f'Best MRR={pd.Series(mrr_tf).max():.4f} achieved with context {pd.Series(mrr_tf).idxmax()[0]} and mlrank = {pd.Series(mrr_tf).idxmax()[1]} and scale factor = {pd.Series(mrr_tf).idxmax()[2]}')
    print(f'Best MRR_pos={pd.Series(mrr_pos_tf).max():.4f} achieved with context {pd.Series(mrr_pos_tf).idxmax()[0]} and mlrank = {pd.Series(mrr_pos_tf).idxmax()[1]} and scale factor = {pd.Series(mrr_pos_tf).idxmax()[2]}')
    print(f'Best MRR_neg={pd.Series(mrr_neg_tf).min():.4f} achieved with context {pd.Series(mrr_neg_tf).idxmin()[0]} and mlrank = {pd.Series(mrr_neg_tf).idxmin()[1]} and scale factor = {pd.Series(mrr_neg_tf).idxmin()[2]}')
    
    print(f'Best Matthews={pd.Series(C_tf).max():.4f} achieved with context {pd.Series(C_tf).idxmax()[0]} and mlrank = {pd.Series(C_tf).idxmax()[1]} and scale factor = {pd.Series(C_tf).idxmax()[2]}')
                          
    print(f'COV={pd.Series(cov_tf)[pd.Series(C_tf).idxmax()]:.4f} (based on best Matthews value)')
    print("---------------------------------------------------------")
    print("Evaluation of the best model on test holdout in progress...\n")
    
    print("Best by MRR@10:\n")
    config["mlrank"] = pd.Series(mrr_pos_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(mrr_pos_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(mrr_pos_tf).idxmax()[0])
    
    print("---------------------------------------------------------")
    
    print("Best by HR@10:\n")
    config["mlrank"] = pd.Series(hr_pos_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(hr_pos_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(hr_pos_tf).idxmax()[0])
    
    print("---------------------------------------------------------")
    
    print("Best by Matthews@10:\n")
    config["mlrank"] = pd.Series(C_tf).idxmax()[1]
    tf_params = tf_model_build(config, training, data_description, testset, holdout, attention_matrix=attention_matrix)

    seen_data = testset
    tf_scores = tf_scoring(tf_params, seen_data, data_description, pd.Series(C_tf).idxmax()[0])
    downvote_seen_items(tf_scores, seen_data, data_description)
    cur_mrr, cur_hr, cur_C = make_prediction(tf_scores, holdout, data_description, "Test", pd.Series(C_tf).idxmax()[0])
    print("Pipeline ended.")

# Custom attention

In [16]:
similarity_matrix = np.array([[1.0, 0.85, 0.2, 0.01, 0.01], [0.85, 1.0, 0.7, 0.2, 0.01], [0.2, 0.7, 1.0, 0.65, 0.5], [0.01, 0.2, 0.65, 1.0, 0.85], [0.01, 0.01, 0.5, 0.85, 1.0]])
similarity_matrix

array([[1.  , 0.85, 0.2 , 0.01, 0.01],
       [0.85, 1.  , 0.7 , 0.2 , 0.01],
       [0.2 , 0.7 , 1.  , 0.65, 0.5 ],
       [0.01, 0.2 , 0.65, 1.  , 0.85],
       [0.01, 0.01, 0.5 , 0.85, 1.  ]])

In [17]:
config["params"] = {}
attention_matrix = sqrtm(similarity_matrix).real
#without sigmoid
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation (Validation): 

HR@5 = 0.0422, MRR@5 = 0.0197, Coverage@5 = 0.1199
HR_pos@5 = 0.0411, HR_neg@5 = 0.0012
MRR_pos@5 = 0.0194, MRR_neg@5 = 0.0003
Matthews@5 = 0.0787
-------------------------------------
HR@10 = 0.0610, MRR@10 = 0.0222, Coverage@10 = 0.1670
HR_pos@10 = 0.0592, HR_neg@10 = 0.0018
MRR_pos@10 = 0.0218, MRR_neg@10 = 0.0003
Matthews@10 = 0.0947
-------------------------------------
HR@20 = 0.1138, MRR@20 = 0.0258, Coverage@20 = 0.2278
HR_pos@20 = 0.1044, HR_neg@20 = 0.0094
MRR_pos@20 = 0.0249, MRR_neg@20 = 0.0008
Matthews@20 = 0.0818
-------------------------------------
------------------------------------------------------
for context 4+5 evaluation (Validation): 

HR@5 = 0.0370, MRR@5 = 0.0197, Coverage@5 = 0.1280
HR_pos@5 = 0.0352, HR_neg@5 = 0.0018
MRR_pos@5 = 0.0183, MRR_neg@5 = 0.0015
Matthews@5 = 0.0630
--------------------

100%|██████████| 420/420 [2:28:13<00:00, 21.17s/it]  


Best HR=0.0997 achieved with context 2+3+4+5 and mlrank = (200, 200, 4) and scale factor = 0.7000000000000001
Best HR_pos=0.0856 achieved with context 2+3+4+5 and mlrank = (200, 200, 4) and scale factor = 0.8
Best HR_neg=0.0023 achieved with context 5 and mlrank = (175, 175, 2) and scale factor = 2.0
Best MRR=0.0357 achieved with context 2+3+4+5 and mlrank = (125, 125, 5) and scale factor = 0.0
Best MRR_pos=0.0329 achieved with context 3+4+5 and mlrank = (175, 175, 3) and scale factor = 0.0
Best MRR_neg=0.0004 achieved with context 5 and mlrank = (100, 100, 4) and scale factor = 0.8
Best Matthews=0.1120 achieved with context 5 and mlrank = (100, 100, 5) and scale factor = 0.9
COV=0.2312 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 3+4+5 evaluation (Test): 

HR@5 = 0.0362, MRR@5 = 0.0175, Coverage@5 = 0.2161
HR_pos@5 = 0.0305, HR_neg@5 = 0.0058
MRR_pos@5 

In [18]:
def tf_scoring(params, data, data_description, context=["3+4+5"]):
    user_factors, item_factors, feedback_factors, attention_matrix = params
    userid = data_description["users"]
    itemid = data_description["items"]
    feedback = data_description["feedback"]

    data = data.sort_values(userid)
    useridx = data[userid]
    itemidx = data[itemid].values
    ratings = data[feedback].values
    ratings = ratings - data_description['min_rating']
    
    n_users = useridx.nunique()
    n_items = data_description['n_items']
    n_ratings = data_description['n_ratings']
    
    inv_attention = solve_triangular(attention_matrix, np.eye(5), lower=True)
    
    tensor_outer = tensor_outer_at('cpu')
    matrix_softmax = softmax(inv_attention.T @ feedback_factors)
    #matrix_softmax = inv_attention.T @ feedback_factors
    #
    if (context == "5"): # make softmax 
        inv_aT_feedback = matrix_softmax[-1, :]
    elif (context == "4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[-2:, :], axis=0)
    elif (context == "3+4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[-3:, :], axis=0)
    elif (context == "2+3+4+5"):
        inv_aT_feedback = np.sum(matrix_softmax[-4:, :], axis=0)
    elif (context == "3+4+5-2-1"):
        inv_aT_feedback = np.sum(matrix_softmax[-3:, :], axis=0) - np.sum(matrix_softmax[:2, :], axis=0)
        
    scores = tensor_outer(
        1.0,
        item_factors,
        attention_matrix @ feedback_factors,
        itemidx,
        ratings
    )
    scores = np.add.reduceat(scores, np.r_[0, np.where(np.diff(useridx))[0]+1]) # sort by users
    scores = np.tensordot(
        scores,
        inv_aT_feedback,
        axes=(2, 0)
    ).dot(item_factors.T)

    return scores

In [19]:
config["params"] = {}
#with sigmoid
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix=attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------
for context 5 evaluation (Validation): 

HR@5 = 0.0358, MRR@5 = 0.0186, Coverage@5 = 0.1160
HR_pos@5 = 0.0358, HR_neg@5 = 0.0000
MRR_pos@5 = 0.0186, MRR_neg@5 = 0.0000
Matthews@5 = 0.0865
-------------------------------------
HR@10 = 0.0598, MRR@10 = 0.0218, Coverage@10 = 0.1559
HR_pos@10 = 0.0581, HR_neg@10 = 0.0018
MRR_pos@10 = 0.0216, MRR_neg@10 = 0.0002
Matthews@10 = 0.0934
-------------------------------------
HR@20 = 0.1114, MRR@20 = 0.0253, Coverage@20 = 0.2131
HR_pos@20 = 0.1062, HR_neg@20 = 0.0053
MRR_pos@20 = 0.0249, MRR_neg@20 = 0.0004
Matthews@20 = 0.1141
-------------------------------------
------------------------------------------------------
for context 4+5 evaluation (Validation): 

HR@5 = 0.0346, MRR@5 = 0.0175, Coverage@5 = 0.1157
HR_pos@5 = 0.0346, HR_neg@5 = 0.0000
MRR_pos@5 = 0.0175, MRR_neg@5 = 0.0000
Matthews@5 = 0.0850
--------------------

100%|██████████| 420/420 [2:50:41<00:00, 24.38s/it]  


Best HR=0.0974 achieved with context 5 and mlrank = (175, 175, 4) and scale factor = 1.1
Best HR_pos=0.0856 achieved with context 5 and mlrank = (200, 200, 4) and scale factor = 1.4000000000000001
Best HR_neg=0.0029 achieved with context 3+4+5-2-1 and mlrank = (125, 125, 3) and scale factor = 1.9000000000000001
Best MRR=0.0359 achieved with context 5 and mlrank = (175, 175, 3) and scale factor = 0.0
Best MRR_pos=0.0319 achieved with context 3+4+5-2-1 and mlrank = (200, 200, 5) and scale factor = 0.2
Best MRR_neg=0.0005 achieved with context 5 and mlrank = (100, 100, 2) and scale factor = 1.8
Best Matthews=0.0988 achieved with context 3+4+5-2-1 and mlrank = (200, 200, 5) and scale factor = 1.9000000000000001
COV=0.2669 (based on best Matthews value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...

Best by MRR@10:

for context 3+4+5-2-1 evaluation (Test): 

HR@5 = 0.0380, MRR@5 = 0.0165, Coverage@5 = 0.2315
HR_pos@5 = 

In [ ]:
# test sigmoid without normalization (scale factor)

## Rating distribution attention

In [10]:
rating_dist = []

total_cnt = training.shape[0]

for i in range(5):
    val = training.query(f'rating == {i + 1}').count()[0] / total_cnt
    
    rating_dist.append(val)

rating_dist

[0.056644931644931645,
 0.10500693000693001,
 0.25943898443898444,
 0.3442087192087192,
 0.2347004347004347]

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        rat_dist_matrix[i, j] = rating_dist[i] / abs(rating_dist[i] - rating_dist[j])
        #rat_dist_matrix[i, j] = diff / np.exp(diff) if i != j else 1. + 1e-1

rat_dist_matrix

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:5: RuntimeWarning: divide by zero encountered in double_scalars
  """


array([[        inf,  1.17126946,  0.27932245,  0.19698214,  0.31813076],
       [ 2.17126946,         inf,  0.67995553,  0.4389889 ,  0.8096545 ],
       [ 1.27932245,  1.67995553,         inf,  3.06051429, 10.48723499],
       [ 1.19698214,  1.4389889 ,  4.06051429,         inf,  3.14322081],
       [ 1.31813076,  1.8096545 ,  9.48723499,  2.14322081,         inf]])

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        diff = abs(rating_dist[i] - rating_dist[j])
        rat_dist_matrix[i, j] = diff / np.exp(diff) if i != j else 1. + 1e-1
        
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)
#rat_dist_matrix

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:02, 445.40it/s]


for context 5 evaluation:
Test : HR@5 = 0.0237, MRR@5 = 0.0144, Coverage@5 = 0.1117, nDCG@5 = 0.016772111754940346, nDCL@5 = 0.0
Test : HR@10 = 0.0466, MRR@10 = 0.0175, Coverage@10 = 0.1546, nDCG@10 = 0.023598806239983595, nDCL@10 = 0.0005780450156306099
Test : HR@20 = 0.0783, MRR@20 = 0.0197, Coverage@20 = 0.2036, nDCG@20 = 0.03118319086912337, nDCL@20 = 0.0010092849309640722
------------------------------------------------------


1137it [00:02, 421.86it/s]


for context 4+5 evaluation:
Test : HR@5 = 0.0334, MRR@5 = 0.0200, Coverage@5 = 0.1109, nDCG@5 = 0.02287084881401924, nDCL@5 = 0.00043975373790677223
Test : HR@10 = 0.0642, MRR@10 = 0.0240, Coverage@10 = 0.1499, nDCG@10 = 0.03215653999889215, nDCL@10 = 0.001030494339396439
Test : HR@20 = 0.1091, MRR@20 = 0.0270, Coverage@20 = 0.2014, nDCG@20 = 0.042042408086102726, nDCL@20 = 0.002324415387296944
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Test : HR@5 = 0.0334, MRR@5 = 0.0172, Coverage@5 = 0.1144, nDCG@5 = 0.02074601094985069, nDCL@5 = 0.0003787832524831953
Test : HR@10 = 0.0712, MRR@10 = 0.0221, Coverage@10 = 0.1552, nDCG@10 = 0.03205373289389733, nDCL@10 = 0.0012005394879658672
Test : HR@20 = 0.1214, MRR@20 = 0.0255, Coverage@20 = 0.2028, nDCG@20 = 0.04261636016198688, nDCL@20 = 0.0031542602033563837
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [06:30<00:00, 10.84s/it]


Best HR=0.0844 achieved with mlrank=(50, 45, 5)
Best MRR=0.0306 achieved with mlrank=(55, 55, 5)
COV=0.1772 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:03, 344.26it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0317, MRR@5 = 0.0143, Coverage@5 = 0.1348, nDCG@5 = 0.016939382722254145, nDCL@5 = 0.0016141705594669602
Test : HR@10 = 0.0580, MRR@10 = 0.0179, Coverage@10 = 0.1774, nDCG@10 = 0.02469648307522927, nDCL@10 = 0.0024464502508388963
Test : HR@20 = 0.1055, MRR@20 = 0.0211, Coverage@20 = 0.2333, nDCG@20 = 0.034409123222778414, nDCL@20 = 0.004649546849046022
Pipeline ended.


## Trigonometry scale attention

In [ ]:
def rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x)

In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = rescale_score(i + 1), rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = diff / np.exp(diff) if i != j else 5 + 1e-2
        
a = np.linalg.cholesky(eucl_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1441.94it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0343, MRR@5 == 0.0208, Coverage@5 = 0.1048
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1461
Validation : HR@20 = 0.1038, MRR@20 == 0.0271, Coverage@20 = 0.2014
------------------------------------------------------


1137it [00:01, 1129.38it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0308, MRR@5 == 0.0183, Coverage@5 = 0.1103
Validation : HR@10 = 0.0633, MRR@10 == 0.0228, Coverage@10 = 0.1475
Validation : HR@20 = 0.1108, MRR@20 == 0.0260, Coverage@20 = 0.2003
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0009, MRR@5 == 0.0003, Coverage@5 = 0.1653
Validation : HR@10 = 0.0018, MRR@10 == 0.0004, Coverage@10 = 0.2360
Validation : HR@20 = 0.0026, MRR@20 == 0.0005, Coverage@20 = 0.3249
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:58<00:00,  6.62s/it]


Best HR=0.0844 achieved with mlrank=(50, 50, 5)
Best MRR=0.0303 achieved with mlrank=(50, 50, 5)
COV=0.1750 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1045.84it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0237, MRR@5 == 0.0103, Coverage@5 = 0.1287
Test : HR@10 = 0.0589, MRR@10 == 0.0150, Coverage@10 = 0.1744
Test : HR@20 = 0.0994, MRR@20 == 0.0177, Coverage@20 = 0.2338
Pipeline ended.


In [ ]:
def center_and_rescale_score(x, func=None):
    
    if func is None:
        func = np.arctan
    
    return func(x - 3)

In [ ]:
eucl_matrix = np.zeros((5, 5))

for i in range(5):
    for j in range(5):
        
        k, l = center_and_rescale_score(i + 1), center_and_rescale_score(j + 1)
        
        diff = abs(k - l)
        
        eucl_matrix[i, j] = 1 / (diff + 1)

similarity = eucl_matrix
        
print(similarity)
    
a = np.linalg.cholesky(similarity)        

attention_matrix = csr_matrix(a)

[[1.         0.75657241 0.47457495 0.34571609 0.31110998]
 [0.75657241 1.         0.56009915 0.38898453 0.34571609]
 [0.47457495 0.56009915 1.         0.56009915 0.47457495]
 [0.34571609 0.38898453 0.56009915 1.         0.75657241]
 [0.31110998 0.34571609 0.47457495 0.75657241 1.        ]]


In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1482.42it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0290, MRR@5 == 0.0190, Coverage@5 = 0.1111
Validation : HR@10 = 0.0528, MRR@10 == 0.0222, Coverage@10 = 0.1530
Validation : HR@20 = 0.0871, MRR@20 == 0.0245, Coverage@20 = 0.2168
------------------------------------------------------


1137it [00:00, 1424.28it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0281, MRR@5 == 0.0192, Coverage@5 = 0.1103
Validation : HR@10 = 0.0598, MRR@10 == 0.0235, Coverage@10 = 0.1505
Validation : HR@20 = 0.1108, MRR@20 == 0.0270, Coverage@20 = 0.2039
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0180, Coverage@5 = 0.1109
Validation : HR@10 = 0.0572, MRR@10 == 0.0217, Coverage@10 = 0.1527
Validation : HR@20 = 0.1099, MRR@20 == 0.0253, Coverage@20 = 0.2116
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:38<00:00,  6.07s/it]


Best HR=0.0827 achieved with mlrank=(50, 50, 5)
Best MRR=0.0293 achieved with mlrank=(55, 50, 5)
COV=0.1816 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1105.85it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0246, MRR@5 == 0.0129, Coverage@5 = 0.1312
Test : HR@10 = 0.0554, MRR@10 == 0.0168, Coverage@10 = 0.1816
Test : HR@20 = 0.0985, MRR@20 == 0.0198, Coverage@20 = 0.2454
Pipeline ended.


## Conditional prob attention

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 2')
users = train_new_part.userid.unique()
count12_tot = 0
count12_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 2:
     count12_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count12_loc += 1
  count12_tot += count12_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 3')
users = train_new_part.userid.unique()
count13_tot = 0
count13_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count13_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count13_loc += 1
  count13_tot += count13_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 4')
users = train_new_part.userid.unique()
count14_tot = 0
count14_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count14_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count14_loc += 1
  count14_tot += count14_loc

In [ ]:
train_new_part = train_new.query('rating == 1| rating == 5')
users = train_new_part.userid.unique()
count15_tot = 0
count15_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 1 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count15_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 1:
      count15_loc += 1
  count15_tot += count15_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 3')
users = train_new_part.userid.unique()
count23_tot = 0
count23_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 3:
     count23_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count23_loc += 1
  count23_tot += count23_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 4')
users = train_new_part.userid.unique()
count24_tot = 0
count24_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count24_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count24_loc += 1
  count24_tot += count24_loc

In [ ]:
train_new_part = train_new.query('rating == 2| rating == 5')
users = train_new_part.userid.unique()
count25_tot = 0
count25_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 2 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count25_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 2:
      count25_loc += 1
  count25_tot += count25_loc

In [ ]:
train_new_part = train_new.query('rating == 3| rating == 4')
users = train_new_part.userid.unique()
count34_tot = 0
count34_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 4:
     count34_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count34_loc += 1
  count34_tot += count34_loc

In [ ]:
train_new_part = train_new.query('rating == 3| rating == 5')
users = train_new_part.userid.unique()
count35_tot = 0
count35_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 3 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count35_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 3:
      count35_loc += 1
  count35_tot += count35_loc

In [ ]:
train_new_part = train_new.query('rating == 4| rating == 5')
users = train_new_part.userid.unique()
count45_tot = 0
count45_loc = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  train_new_part_user = train_new_part_user.reset_index()
  for i in range(1, len(train_new_part_user)):
    if train_new_part_user.loc[i, 'rating' ] == 4 and train_new_part_user.loc[i-1, 'rating' ]== 5:
     count45_loc += 1
    if train_new_part_user.loc[i, 'rating' ] == 5 and train_new_part_user.loc[i-1, 'rating' ]== 4:
      count45_loc += 1
  count45_tot += count45_loc

In [ ]:
train_new_part = train_new.query('rating == 1')
users = train_new_part.userid.unique()
count11_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count11_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 2')
users = train_new_part.userid.unique()
count22_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count22_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 3')
users = train_new_part.userid.unique()
count33_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count33_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 4')
users = train_new_part.userid.unique()
count44_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count44_tot += len(train_new_part_user)-1

In [ ]:
train_new_part = train_new.query('rating == 5')
users = train_new_part.userid.unique()
count55_tot = 0
for user in users:
  train_new_part_user = train_new_part.query('userid == @user') 
  count55_tot += len(train_new_part_user)-1

In [ ]:
rat_dist_matrix = np.zeros((5, 5))

rat_dist_matrix[0][0] = count11_tot
rat_dist_matrix[0][1] = rat_dist_matrix[1][0]= count12_tot
rat_dist_matrix[0][2] = rat_dist_matrix[2][0]= count13_tot
rat_dist_matrix[0][3] = rat_dist_matrix[3][0]= count14_tot
rat_dist_matrix[0][4] = rat_dist_matrix[4][0]= count15_tot
rat_dist_matrix[1][1] = count22_tot
rat_dist_matrix[1][2] = rat_dist_matrix[2][1] = count23_tot
rat_dist_matrix[1][3] = rat_dist_matrix[3][1] = count24_tot
rat_dist_matrix[1][4] = rat_dist_matrix[4][1] = count25_tot
rat_dist_matrix[2][2] = count33_tot
rat_dist_matrix[2][3] = rat_dist_matrix[3][2] = count34_tot
rat_dist_matrix[2][4] = rat_dist_matrix[4][2] = count35_tot
rat_dist_matrix[3][3] = count44_tot
rat_dist_matrix[3][4] = rat_dist_matrix[4][3] = count45_tot
rat_dist_matrix[4][4] = count55_tot       

In [ ]:
summ = np.sum(rat_dist_matrix)
rat_dist_matrix /= summ

In [ ]:
rat_dist_matrix[0,0] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[1,1] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[2,2] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[3,3] += np.random.uniform(low=0.0, high=1.0)
rat_dist_matrix[4,4] += np.random.uniform(low=0.0, high=1.0)

In [ ]:
a = np.linalg.cholesky(rat_dist_matrix)

for i in range(5):
    a[i, i] = 1e-5

attention_matrix = csr_matrix(a)

In [ ]:
full_pipeline(config, training, data_description, testset_valid, holdout_valid, testset, holdout, attention_matrix)

Starting pipeline...
Training with different context in progress...
------------------------------------------------------


1137it [00:00, 1542.55it/s]


for context 5 evaluation:
Validation : HR@5 = 0.0299, MRR@5 == 0.0164, Coverage@5 = 0.0949
Validation : HR@10 = 0.0554, MRR@10 == 0.0195, Coverage@10 = 0.1337
Validation : HR@20 = 0.0880, MRR@20 == 0.0217, Coverage@20 = 0.1796
------------------------------------------------------


1137it [00:00, 1359.75it/s]


for context 4+5 evaluation:
Validation : HR@5 = 0.0317, MRR@5 == 0.0188, Coverage@5 = 0.0988
Validation : HR@10 = 0.0598, MRR@10 == 0.0226, Coverage@10 = 0.1309
Validation : HR@20 = 0.0950, MRR@20 == 0.0250, Coverage@20 = 0.1772
------------------------------------------------------


...

for context 3+4+5-2-1 evaluation:
Validation : HR@5 = 0.0325, MRR@5 == 0.0203, Coverage@5 = 0.1004
Validation : HR@10 = 0.0624, MRR@10 == 0.0244, Coverage@10 = 0.1304
Validation : HR@20 = 0.1029, MRR@20 == 0.0272, Coverage@20 = 0.1769
------------------------------------------------------
Tuning model with context 3+4+5...


100%|██████████| 36/36 [03:55<00:00,  6.54s/it]


Best HR=0.0800 achieved with mlrank=(45, 45, 5)
Best MRR=0.0279 achieved with mlrank=(40, 45, 5)
COV=0.1560 (based on best HR value)
---------------------------------------------------------
Evaluation of the best model on test holdout in progress...


1137it [00:01, 1100.26it/s]


for context 3+4+5 evaluation:
Test : HR@5 = 0.0290, MRR@5 == 0.0126, Coverage@5 = 0.1161
Test : HR@10 = 0.0528, MRR@10 == 0.0157, Coverage@10 = 0.1549
Test : HR@20 = 0.1038, MRR@20 == 0.0190, Coverage@20 = 0.2077
Pipeline ended.
